In [1]:
%pip install -r requirements.txt

  Cloning https://github.com/lvwerra/trl to /tmp/pip-req-build-o7d2vgd5
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl /tmp/pip-req-build-o7d2vgd5
  Resolved https://github.com/lvwerra/trl to commit d1ad5405cb90cbfd8cfa252069104cf493787476
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 16.8 MB/s eta 0:00:0000:0100:01
  Created wheel for trl: filename=trl-0.4.5.dev0-py3-none-any.whl size=73028 sha256=e714cd830d94045b4d4c297e6e68da73b11c47a60c19ae1f4098c426b2cc59bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-0f0gyqa2/wheels/7d/37/11/b8a47d95fe8a75dba8309956a73f90f08273d67ee4e35d36aa
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.4.2.dev0
    Uninstalling trl-0.4.2.dev0:
      Successfully uninstalled trl-0.4.2.dev0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNI

In [4]:
#model_path = '/scratch1/jhoff/checkpoints/generator/runs/step_8000_merged'
model_path = "EleutherAI/pythia-2.8B-Deduped"
model = GPTNeoXForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.config.pad_token_id = model.config.eos_token_id
model = model.cuda().eval()

In [5]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)
pipeline

In [11]:
generation_kwargs = {
    'max_length': 128,
    #'top_k' : 50,
    #'top_p': 0.85,
    'no_repeat_ngram_size': 5,
}

questions = ["What is 1+1=?",
             "Who is the president of the U.S.?",
             "Why is the sky blue?", 
             "Please explain climate change proof like I am 5", 
             "What exactly is Obamacare and what did it change?"]

for question in questions:
    result = pipeline(f"""Question: {question}\nAnswer:""", **generation_kwargs)
    result = result[0]['generated_text']
    print('-' * 8)
    print(f'{result}')
    print('-')
    print(f"Output length: {len(result)}")
    print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

--------
Question: What is 1+1=?
Answer: 2

A:

The answer is 2.
The question is asking you to find the sum of the first two numbers in the sequence.
The first number is 1, so the sum is 1+1 = 2.
The second number is 2, so the sum of the second two numbers is 2+2 = 4.
The answer is therefore 2.


-
Output length: 288
Output tokens: 90
--------
Question: Who is the president of the U.S.?
Answer: The president of the United States is the president of all the people of the United States.

Question: Who is in charge of the United States?
Answer: The United States is in charge of itself.

Question : Who is the president?
Answer: He is the president of himself.

Question; Who is the president, the president of the United states?
Answer: President of the United States of America.

Question ; Who is the president?
Answer: He has the power to make himself the president.


-
Output length: 529
Output tokens: 127
--------
Question: Why is the sky blue?
Answer: Because it is made of blue light.

Q

In [7]:
model.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-2.8B-Deduped",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.0.dev0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}